<a href="https://colab.research.google.com/github/morioka/BERT_Japanese_Google_Colaboratory/blob/master/2_BERT_livedoor_news_on_Google_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 日本語BERTでlivedoorニュースを教師あり学習で分類

In [1]:
# 乱数シードの固定

import os
import random
import numpy as np
import torch

SEED_VALUE = 1234  # これはなんでも良い
os.environ['PYTHONHASHSEED'] = str(SEED_VALUE)
random.seed(SEED_VALUE)
np.random.seed(SEED_VALUE)
torch.manual_seed(SEED_VALUE)  # PyTorchを使う場合


### GPUの使用可能を確認

画面上部のメニュー ランタイム > ランタイムのタイプを変更 で、 ノートブックの設定 を開く

ハードウェアアクセラレータに GPU を選択し、 保存 する

In [2]:
# GPUの使用確認：True or False
torch.cuda.is_available()

# TrueならGPU使用可能

True

## 準備1：Livedoorニュースをダウンロードしてtsvファイル化

参考：https://github.com/yoheikikuta/bert-japanese/blob/master/notebook/finetune-to-livedoor-corpus.ipynb


In [3]:
# Livedoorニュースのファイルをダウンロード
! wget "https://www.rondhuit.com/download/ldcc-20140209.tar.gz"

--2021-04-23 15:22:15--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
Resolving www.rondhuit.com (www.rondhuit.com)... 59.106.19.174
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8855190 (8.4M) [application/x-gzip]
Saving to: ‘ldcc-20140209.tar.gz’

ldcc-20140209.tar.g 100%[===================>]   8.44M  1.86MB/s    in 5.0s    

2021-04-23 15:22:21 (1.70 MB/s) - ‘ldcc-20140209.tar.gz’ saved [8855190/8855190]



In [4]:
# ファイルを解凍し、カテゴリー数と内容を確認
import tarfile
import os

# 解凍
tar = tarfile.open("ldcc-20140209.tar.gz", "r:gz")
tar.extractall("./data/livedoor/")
tar.close()

# フォルダのファイルとディレクトリを確認
files_folders = [name for name in os.listdir("./data/livedoor/text/")]
print(files_folders)

# カテゴリーのフォルダのみを抽出
categories = [name for name in os.listdir(
    "./data/livedoor/text/") if os.path.isdir("./data/livedoor/text/"+name)]

print("カテゴリー数:", len(categories))
print(categories)


['peachy', 'movie-enter', 'smax', 'kaden-channel', 'livedoor-homme', 'it-life-hack', 'topic-news', 'sports-watch', 'README.txt', 'dokujo-tsushin', 'CHANGES.txt']
カテゴリー数: 9
['peachy', 'movie-enter', 'smax', 'kaden-channel', 'livedoor-homme', 'it-life-hack', 'topic-news', 'sports-watch', 'dokujo-tsushin']


In [5]:
# ファイルの中身を確認してみる
file_name = "./data/livedoor/text/movie-enter/movie-enter-6255260.txt"

with open(file_name) as text_file:
    text = text_file.readlines()
    print("0：", text[0])  # URL情報
    print("1：", text[1])  # タイムスタンプ
    print("2：", text[2])  # タイトル
    print("3：", text[3])  # 本文

    # 今回は4要素目には本文は伸びていないが、4要素目以降に本文がある場合もある


0： http://news.livedoor.com/article/detail/6255260/

1： 2012-02-07T09:00:00+0900

2： 新しいヴァンパイアが誕生！　ジョニデ主演『ダーク・シャドウ』の公開日が決定

3： 　こんなヴァンパイアは見たことがない！　ジョニー・デップとティム・バートン監督がタッグを組んだ映画『ダーク・シャドウズ（原題）』の邦題が『ダーク・シャドウ』に決定。日本公開日が5月19日に決まった。さらに、ジョニー・デップ演じるヴァンパイアの写真が公開された。



In [6]:
# 本文を取得する前処理関数を定義


def extract_main_txt(file_name):
    with open(file_name) as text_file:
        # 今回はタイトル行は外したいので、3要素目以降の本文のみ使用
        text = text_file.readlines()[3:]

        # 3要素目以降にも本文が入っている場合があるので、リストにして、後で結合させる
        text = [sentence.strip() for sentence in text]  # 空白文字(スペースやタブ、改行)の削除
        text = list(filter(lambda line: line != '', text))
        text = ''.join(text)
        text = text.translate(str.maketrans(
            {'\n': '', '\t': '', '\r': '', '\u3000': ''}))  # 改行やタブ、全角スペースを消す
        return text


In [7]:
# リストに前処理した本文と、カテゴリーのラベルを追加していく
import glob

list_text = []
list_label = []

for cat in categories:
    text_files = glob.glob(os.path.join("./data/livedoor/text", cat, "*.txt"))

    # 前処理extract_main_txtを実施して本文を取得
    body = [extract_main_txt(text_file) for text_file in text_files]

    label = [cat] * len(body)  # bodyの数文だけカテゴリー名のラベルのリストを作成

    list_text.extend(body)  # appendが要素を追加するのに対して、extendはリストごと追加する
    list_label.extend(label)


In [8]:
# 0番目の文章とラベルを確認
print(list_text[0])
print(list_label[0])


世の多くの女性をとりこにする「チョコレート」。実は、私達「才職兼美」の編集女子部員も、チョコレートが大好きなんです。締め切りに追われる日々の疲れた身体を癒してくれる、甘い幸せ…。そこで、編集女子部員が、個人的にこれぞと思う魅惑の逸品をご紹介します！＜編集部員Fのオススメ！＞■カー・ヴァンソン「トリュフ」以前、神楽坂にいた時、近所に開店したパティスリーがここだったんです。特にチョコレートが絶品！塩味に効いたショコラキャラメルや口どけの優しいトリュフは、幸せをくれる味。だから、特別な時にはここのチョコレートをおもたせやプレゼントにしています。もちろん、バレンタインチョコにも愛用。ただ、さすがに義理用にはもったいないので、本命用として利用しています。・パティスリー カー・ヴァンソン＜編集部員Mのオススメ！＞■ミルカ「ハッピーカウ」ドイツのブランド「ミルカ（Milka）」のチョコレート。お菓子には珍しいライラック色のパッケージがキレイです。一時期は、日本のスーパーで購入でき、TVCMも流れるほどでしたが、今は取り扱い店が少なく、寂しい限り…。輸入食品ショップなどで見かけると、必ず買ってしまいます。この「ハッピーカウ（Happy Cows）」は、その名の通り、ミルクチョコレートとホワイトチョコレートが混ざって可愛らしい牛柄（白と黒が反転していますが）になっています。ミルク感たっぷりで、クセのない、優しい味。仕事の合間に食べると、ホッと優しい気持ちになれちゃいます。・ミルカ■コートドール「タブレットノアーデノアー」ベルギーのブランド「コートドール（Cote d'Or）」のチョコレート。パッケージのデザインが歴史を感じさせ、西アフリカの地名「黄金海岸」から取ったというブランド名や、純白の象のシンボルマークがエキゾチックです。ミルクやビターといった味のバリエーション、また、タブレットやミニョネッツ（ミニチュアサイズ）と形状も豊富で、選び甲斐があります。この「ノアーデノアー」の味は「エクストラビター」とのこと。とはいえ、カカオ分56%と高すぎないので、それほど苦くなく、甘さとカカオ感のバランスがちょうど良いです。たっぷり2枚入りなのも、「お得！」と、嬉しい気持ちになりませんか？・コートドール＜編集部員Kのオススメ！＞■ピエール・エルメ・パリ「キャレ・ショコラ・オウ・レ」私のお気に入り

In [9]:
# pandasのDataFrameにする
import pandas as pd

df = pd.DataFrame({'text': list_text, 'label': list_label})

# 大きさを確認しておく（7,376文章が存在）
print(df.shape)

df.head()


(7376, 2)


,text,label
0,世の多くの女性をとりこにする「チョコレート」。実は、私達「才職兼美」の編集女子部員も、チョコ...,peachy
1,【スナップレポート】東京ガールズコレクション 2011 土屋アンナ編をもっと見る,peachy
2,これからの季節、オフィスや電車の冷房で意外に冷えてしまうことってありますよね、そんな時一枚持...,peachy
3,ロマンチックなクリスマス・・・を過ごしたのはいつの頃でしょう？最近、彼とのデートは、カフェで...,peachy
4,女子なら誰でも子供時代に夢中になったファッションドールの「バービー人形」。皆さんも抜群のプロ...,peachy


In [10]:
# カテゴリーの辞書を作成
dic_id2cat = dict(zip(list(range(len(categories))), categories))
dic_cat2id = dict(zip(categories, list(range(len(categories)))))

print(dic_id2cat)
print(dic_cat2id)

# DataFrameにカテゴリーindexの列を作成
df["label_index"] = df["label"].map(dic_cat2id)
df.head()

# label列を消去し、text, indexの順番にする
df = df.loc[:, ["text", "label_index"]]
df.head()


{0: 'peachy', 1: 'movie-enter', 2: 'smax', 3: 'kaden-channel', 4: 'livedoor-homme', 5: 'it-life-hack', 6: 'topic-news', 7: 'sports-watch', 8: 'dokujo-tsushin'}
{'peachy': 0, 'movie-enter': 1, 'smax': 2, 'kaden-channel': 3, 'livedoor-homme': 4, 'it-life-hack': 5, 'topic-news': 6, 'sports-watch': 7, 'dokujo-tsushin': 8}


,text,label_index
0,世の多くの女性をとりこにする「チョコレート」。実は、私達「才職兼美」の編集女子部員も、チョコ...,0
1,【スナップレポート】東京ガールズコレクション 2011 土屋アンナ編をもっと見る,0
2,これからの季節、オフィスや電車の冷房で意外に冷えてしまうことってありますよね、そんな時一枚持...,0
3,ロマンチックなクリスマス・・・を過ごしたのはいつの頃でしょう？最近、彼とのデートは、カフェで...,0
4,女子なら誰でも子供時代に夢中になったファッションドールの「バービー人形」。皆さんも抜群のプロ...,0


In [11]:
# 順番をシャッフルする
df = df.sample(frac=1, random_state=123).reset_index(drop=True)
df.head()

,text,label_index
0,ソフトバンクグループの代表 孫正義氏は、Twitterを通じて活発な発言をしている。そんな同...,5
1,1日、フジテレビのバラエティー番組「あいのり」に出演していた東京都渋谷区の会社員・南里始(な...,6
2,オーストラリアでタブレット端末の販売差し止めの仮処分取り消しを求めていたサムスン。アップルに...,3
3,30日、甲子園で行われたプロ野球公式戦＝阪神×ロッテの試合において、中継を担当した神戸のUH...,6
4,誰でも何かを購入するときは商品を調べるだろう。高額になればなおさら、商品をよく知らずに購入は...,5


In [12]:
# tsvファイルで保存する

# 全体の2割の文章数
len_0_2 = len(df) // 5

# 前から2割をテストデータとする
df[:len_0_2].to_csv("./test.tsv", sep='\t', index=False, header=None)
print(df[:len_0_2].shape)

# 前2割からを訓練&検証データとする
df[len_0_2:].to_csv("./train_eval.tsv", sep='\t', index=False, header=None)
print(df[len_0_2:].shape)


(1475, 2)
(5901, 2)


In [13]:
# tsvファイルをダウンロードしたい場合
from google.colab import files

# ダウンロードする場合はコメントを外す
# 少し時間がかかる（4MB）
# files.download("./test.tsv")


# ダウンロードする場合はコメントを外す
# 少し時間がかかる（18MB）
# files.download("./train_eval.tsv")


## 準備2：LivedoorニュースをBERT用のDataLoaderにする

Hugginfaceのリポジトリの案内とは異なり、torchtextを使用した手法で実装

In [14]:
# MeCabとtransformersの用意
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3
!pip install transformers==4.5.1
!pip install fugashi ipadic

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
  swig3.0
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl lib

In [15]:
!ln -s /etc/mecabrc /usr/local/etc/mecabrc

In [16]:
import torch
import torchtext.legacy as torchtext  # current 0.9.1 move Fields, Examples to legacy.
from transformers import BertModel
from transformers import BertJapaneseTokenizer

# 日本語BERTの分かち書き用tokenizerです
tokenizer = BertJapaneseTokenizer.from_pretrained(
    'cl-tohoku/bert-base-japanese-whole-word-masking')


In [17]:
# データを読み込んだときに、読み込んだ内容に対して行う処理を定義します

max_length = 512  # 東北大学_日本語版の最大の単語数（サブワード数）は512


def tokenizer_512(input_text):
    """torchtextのtokenizerとして扱えるように、512単語のpytorchでのencodeを定義。ここで[0]を指定し忘れないように"""
    return tokenizer.encode(input_text, max_length=512, return_tensors='pt')[0]


TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_512, use_vocab=False, lower=False,
                            include_lengths=True, batch_first=True, fix_length=max_length, pad_token=0)
# 注意：tokenize=tokenizer.encodeと、.encodeをつけます。padding[PAD]のindexが0なので、0を指定します。

LABEL = torchtext.data.Field(sequential=False, use_vocab=False)

# (注釈)：各引数を再確認
# sequential: データの長さが可変か？文章は長さがいろいろなのでTrue.ラベルはFalse
# tokenize: 文章を読み込んだときに、前処理や単語分割をするための関数を定義
# use_vocab：単語をボキャブラリーに追加するかどうか
# lower：アルファベットがあったときに小文字に変換するかどうか
# include_length: 文章の単語数のデータを保持するか
# batch_first：ミニバッチの次元を用意するかどうか
# fix_length：全部の文章をfix_lengthと同じ長さになるように、paddingします
# init_token, eos_token, pad_token, unk_token：文頭、文末、padding、未知語に対して、どんな単語を与えるかを指定


In [18]:
# 各tsvファイルを読み込み、分かち書きをしてdatasetにします
# 少し時間がかかります
# train_eval：5901個、test：1475個
dataset_train_eval, dataset_test = torchtext.data.TabularDataset.splits(
    path='.', train='train_eval.tsv', test='test.tsv', format='tsv', fields=[('Text', TEXT), ('Label', LABEL)])



Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [19]:
# torchtext.data.Datasetのsplit関数で訓練データと検証データを分ける
# train_eval：5901個、test：1475個

dataset_train, dataset_eval = dataset_train_eval.split(
    split_ratio=1.0 - 1475/5901, random_state=random.seed(1234))

# datasetの長さを確認してみる
print(dataset_train.__len__())
print(dataset_eval.__len__())
print(dataset_test.__len__())


4426
1475
1475


In [20]:
# datasetの中身を確認してみる
item = next(iter(dataset_train))
print(item.Text)
print("長さ：", len(item.Text))  # 長さを確認 [CLS]から始まり[SEP]で終わる。512より長いと後ろが切れる
print("ラベル：", item.Label)


tensor([    2,    48,    32,     6,  3205,  5640,    12,   343,    20,    10,
            6,  1301,   472,  3411,  2185,  1140,  2684,    12,     6,    91,
          542,     9, 12282,   339,   542,    11,  1879,     7,    48,    61,
          518,    12,   610, 28724,     8, 17193,     5,  3936,    36,  2087,
         3594, 28454,   571,    38,     7,     9,     6,  1301,  3226,   104,
           35,   812, 28906,   497,  2976,   643,    14,  3453,   793,    11,
         1521,     6,   608,    11, 17472,    10,     8,  3001,     5,  7353,
          221,     6,   812, 28906,   643,     9,    36,    52, 28562,   219,
            7,    15,    16,     9, 10821,    48,  3028, 11646,    10,    45,
           14,  3370,     8, 25482,  1281,    16,    28,  2105,     5,   640,
         1713,    14, 22752,    10,    38,    36,  7755,   575,   308,  3318,
            6,  4467,    25,   652,     9,  6628,    10,    38,    64,    13,
        16040, 20236,     7,  8085,    13,     6,  1411,    12, 

In [21]:
# datasetの中身を文章に戻し、確認

print(tokenizer.convert_ids_to_tokens(item.Text.tolist()))  # 文章
dic_id2cat[int(item.Label)]  # id


['[CLS]', '3', '日', '、', '埼玉', 'スタジアム', 'で', '行わ', 'れ', 'た', '、', 'サッカー', 'W', '杯', 'アジア', '最終', '予選', 'で', '、', '日本', '代表', 'は', 'オマ', '##ーン', '代表', 'を', '相手', 'に', '3', '-', '0', 'で', '完', '##勝', '。', '翌朝', 'の', 'フジテレビ', '「', 'め', '##ざま', '##し', 'テレビ', '」', 'に', 'は', '、', 'サッカー', '解説', '者', '・', '松', '##木', '安', '##太郎', '氏', 'が', 'ゲスト', '出演', 'を', '果たし', '、', '試合', 'を', '振り返っ', 'た', '。', 'コーナー', 'の', '冒頭', 'より', '、', '松', '##木', '氏', 'は', '「', '一', '##戦', '目', 'に', 'し', 'て', 'は', '本当に', '3', 'ポイント', '取れ', 'た', 'こと', 'が', '大きい', '。', 'なんと', 'いっ', 'て', 'も', '得点', 'の', '時間', '帯', 'が', 'よかっ', 'た', '」', '「', 'あの', 'チーム', 'だっ', 'たら', '、', 'あと', '2', '点', 'は', 'いけ', 'た', '」', 'など', 'と', '興奮', '気味', 'に', '語る', 'と', '、', 'ここ', 'で', 'サポーター', 'から', 'の', '質問', 'が', 'VTR', 'で', '紹介', 'さ', 'れ', 'た', '。', 'まずは', '、', '「', '香川', 'と', '本田', 'は', 'どちら', 'が', 'エース', '?」', 'という', '問い', 'に', '、', '「', '二', '人', 'いる', 'こと', 'で', '分散', 'する', 'ん', 'です', 'よ', '、', '向こう', 'の', 'ディフェンス', 'が', '。', 'どちら', 'が', 

'sports-watch'

In [22]:
# DataLoaderを作成します（torchtextの文脈では単純にiteraterと呼ばれています）
batch_size = 16  # BERTでは16、32あたりを使用する

dl_train = torchtext.data.Iterator(
    dataset_train, batch_size=batch_size, train=True)

dl_eval = torchtext.data.Iterator(
    dataset_eval, batch_size=batch_size, train=False, sort=False)

dl_test = torchtext.data.Iterator(
    dataset_test, batch_size=batch_size, train=False, sort=False)

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": dl_train, "val": dl_eval}


In [23]:
# DataLoaderの動作確認 

batch = next(iter(dl_test))
print(batch)
print(batch.Text[0].shape)
print(batch.Label.shape)


[torchtext.legacy.data.batch.Batch of size 16]
	[.Text]:('[torch.LongTensor of size 16x512]', '[torch.LongTensor of size 16]')
	[.Label]:[torch.LongTensor of size 16]
torch.Size([16, 512])
torch.Size([16])


## 準備3：BERTのクラス分類用のモデルを用意する

Huggingfaceさんのをそのまま使うのではなく、BERTのbaseだけ使い、残りは自分で実装する

In [24]:
from transformers import BertModel

# BERTの日本語学習済みパラメータのモデルです
model = BertModel.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
print(model)



BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(32000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
         

In [25]:
from torch import nn


class BertForLivedoor(nn.Module):
    '''BERTモデルにLivedoorニュースの9クラスを判定する部分をつなげたモデル'''

    def __init__(self):
        super(BertForLivedoor, self).__init__()

        # BERTモジュール
        self.bert = model  # 日本語学習済みのBERTモデル

        # headにポジネガ予測を追加
        # 入力はBERTの出力特徴量の次元768、出力は9クラス
        self.cls = nn.Linear(in_features=768, out_features=9)

        # 重み初期化処理
        nn.init.normal_(self.cls.weight, std=0.02)
        nn.init.normal_(self.cls.bias, 0)

    def forward(self, input_ids):
        '''
        input_ids： [batch_size, sequence_length]の文章の単語IDの羅列
        '''

        # BERTの基本モデル部分の順伝搬
        # 順伝搬させる
        result = self.bert(input_ids)  # reult は、sequence_output, pooled_output

        # sequence_outputの先頭の単語ベクトルを抜き出す
        vec_0 = result[0]  # 最初の0がsequence_outputを示す
        vec_0 = vec_0[:, 0, :]  # 全バッチ。先頭0番目の単語の全768要素
        vec_0 = vec_0.view(-1, 768)  # sizeを[batch_size, hidden_size]に変換
        output = self.cls(vec_0)  # 全結合層

        return output


In [26]:
# モデル構築
net = BertForLivedoor()

# 訓練モードに設定
net.train()

print('ネットワーク設定完了')


ネットワーク設定完了


## 準備4：BERTのファインチューニングの設定

In [27]:
# 勾配計算を最後のBertLayerモジュールと追加した分類アダプターのみ実行

# 1. まず全部を、勾配計算Falseにしてしまう
for param in net.parameters():
    param.requires_grad = False

# 2. BertLayerモジュールの最後を勾配計算ありに変更
for param in net.bert.encoder.layer[-1].parameters():
    param.requires_grad = True

# 3. 識別器を勾配計算ありに変更
for param in net.cls.parameters():
    param.requires_grad = True


In [28]:
# 最適化手法の設定
import torch.optim as optim


# BERTの元の部分はファインチューニング
optimizer = optim.Adam([
    {'params': net.bert.encoder.layer[-1].parameters(), 'lr': 5e-5},
    {'params': net.cls.parameters(), 'lr': 1e-4}
])

# 損失関数の設定
criterion = nn.CrossEntropyLoss()
# nn.LogSoftmax()を計算してからnn.NLLLoss(negative log likelihood loss)を計算

## 5. 訓練を実施

In [29]:
# モデルを学習させる関数を作成


def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)
    print('-----start-------')

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # ミニバッチのサイズ
    batch_size = dataloaders_dict["train"].batch_size

    # epochのループ
    for epoch in range(num_epochs):
        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
            else:
                net.eval()   # モデルを検証モードに

            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数
            iteration = 1

            # データローダーからミニバッチを取り出すループ
            for batch in (dataloaders_dict[phase]):
                # batchはTextとLableの辞書型変数

                # GPUが使えるならGPUにデータを送る
                inputs = batch.Text[0].to(device)  # 文章
                labels = batch.Label.to(device)  # ラベル

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):

                    # BERTに入力
                    outputs = net(inputs)

                    loss = criterion(outputs, labels)  # 損失を計算

                    _, preds = torch.max(outputs, 1)  # ラベルを予測

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                        if (iteration % 10 == 0):  # 10iterに1度、lossを表示
                            acc = (torch.sum(preds == labels.data)
                                   ).double()/batch_size
                            print('イテレーション {} || Loss: {:.4f} || 10iter. || 本イテレーションの正解率：{}'.format(
                                iteration, loss.item(),  acc))

                    iteration += 1

                    # 損失と正解数の合計を更新
                    epoch_loss += loss.item() * batch_size
                    epoch_corrects += torch.sum(preds == labels.data)

            # epochごとのlossと正解率
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloaders_dict[phase].dataset)

            print('Epoch {}/{} | {:^5} |  Loss: {:.4f} Acc: {:.4f}'.format(epoch+1, num_epochs,
                                                                           phase, epoch_loss, epoch_acc))

    return net


In [ ]:
# 学習・検証を実行する。1epochに2分ほどかかります
num_epochs = 4
net_trained = train_model(net, dataloaders_dict,
                          criterion, optimizer, num_epochs=num_epochs)


使用デバイス： cuda:0
-----start-------
イテレーション 10 || Loss: 2.0255 || 10iter. || 本イテレーションの正解率：0.3125
イテレーション 20 || Loss: 1.9916 || 10iter. || 本イテレーションの正解率：0.125
イテレーション 30 || Loss: 1.6376 || 10iter. || 本イテレーションの正解率：0.5625
イテレーション 40 || Loss: 1.5647 || 10iter. || 本イテレーションの正解率：0.5625
イテレーション 50 || Loss: 1.3101 || 10iter. || 本イテレーションの正解率：0.625
イテレーション 60 || Loss: 1.0139 || 10iter. || 本イテレーションの正解率：0.625
イテレーション 70 || Loss: 0.9623 || 10iter. || 本イテレーションの正解率：0.6875
イテレーション 80 || Loss: 0.9325 || 10iter. || 本イテレーションの正解率：0.8125
イテレーション 90 || Loss: 0.4141 || 10iter. || 本イテレーションの正解率：0.9375
イテレーション 100 || Loss: 0.3122 || 10iter. || 本イテレーションの正解率：1.0
イテレーション 110 || Loss: 1.0851 || 10iter. || 本イテレーションの正解率：0.5625
イテレーション 120 || Loss: 0.3104 || 10iter. || 本イテレーションの正解率：1.0
イテレーション 130 || Loss: 0.8922 || 10iter. || 本イテレーションの正解率：0.75
イテレーション 140 || Loss: 0.4443 || 10iter. || 本イテレーションの正解率：0.9375
イテレーション 150 || Loss: 0.5626 || 10iter. || 本イテレーションの正解率：0.875
イテレーション 160 || Loss: 0.6117 || 10iter. || 本イテレーションの正解率：0.6

## テストデータでの性能を確認

In [ ]:
from tqdm import tqdm

# テストデータでの正解率を求める
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net_trained.eval()   # モデルを検証モードに
net_trained.to(device)  # GPUが使えるならGPUへ送る

# epochの正解数を記録する変数
epoch_corrects = 0

for batch in tqdm(dl_test):  # testデータのDataLoader
    # batchはTextとLableの辞書オブジェクト
    # GPUが使えるならGPUにデータを送る
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    inputs = batch.Text[0].to(device)  # 文章
    labels = batch.Label.to(device)  # ラベル

    # 順伝搬（forward）計算
    with torch.set_grad_enabled(False):

        # BertForLivedoorに入力
        outputs = net_trained(inputs)

        loss = criterion(outputs, labels)  # 損失を計算
        _, preds = torch.max(outputs, 1)  # ラベルを予測
        epoch_corrects += torch.sum(preds == labels.data)  # 正解数の合計を更新

# 正解率
epoch_acc = epoch_corrects.double() / len(dl_test.dataset)

print('テストデータ{}個での正解率：{:.4f}'.format(len(dl_test.dataset), epoch_acc))


https://yoheikikuta.github.io/bert-japanese/

https://github.com/yoheikikuta/bert-japanese

の「BERT with SentencePiece for Japanese text.」

では、入力テキストにタイトルを含めていますが、今回はタイトルは除いています。

同様にタイトルを抜いている、[BERTを用いた日本語文書分類タスクの学習・ハイパーパラメータチューニングの実践例](https://medium.com/karakuri/bert%E3%82%92%E7%94%A8%E3%81%84%E3%81%9F%E6%97%A5%E6%9C%AC%E8%AA%9E%E6%96%87%E6%9B%B8%E5%88%86%E9%A1%9E%E3%82%BF%E3%82%B9%E3%82%AF%E3%81%AE%E5%AD%A6%E7%BF%92-%E3%83%8F%E3%82%A4%E3%83%91%E3%83%BC%E3%83%91%E3%83%A9%E3%83%A1%E3%83%BC%E3%82%BF%E3%83%81%E3%83%A5%E3%83%BC%E3%83%8B%E3%83%B3%E3%82%B0%E3%81%AE%E5%AE%9F%E8%B7%B5%E4%BE%8B-2fa5e4299b16)でも、正解率が92%ちょっととなっており、ほぼ同じ正解率が得られました。

以上。